# 📢📢📢 Projet NLP : La parole aux citoyens (Make.org - Le tourisme vert en Ille-et-Vilaine)📢📢📢
# Visualisations et vectorisation des mots
## Bibliothèques nécessaires

In [1]:
import ast
import os
from itertools import chain

import numpy as np
import pandas as pd
import plotly.express as px
from gensim.models import Word2Vec
from PIL import Image
from wordcloud import WordCloud

## Lecture des données

In [2]:
df = pd.read_csv(
    "../data/processed/req_tourisme_responsable_tokenised.csv",
    delimiter=";",
    encoding="utf-8",
)

df.head(3)

,content,agree_count,disagree_count,neutral_count,agree_score,disagree_score,neutral_score,doNotCare,doNotUnderstand,doable,...,platitudeAgree,platitudeDisagree,nb_interrogations,nb_exclamation,tokens,case,tokens_normalized,stop_words,tokens_final,tokens_lemmatized
0,Il faut une surveillance des lieux touristique...,27,7,7,0.65,0.18,0.17,1,0,8,...,3,4,0,0,"['Il', 'faut', 'une', 'surveillance', 'des', '...","['Min', 'Min', 'Min', 'Min', 'Min', 'Min', 'Mi...","['une', 'surveillance', 'des', 'lieux', 'touri...","['il', 'une', 'des', 'avec', 'la', 'pour', 'ou']","['surveillance', 'lieux', 'amende', 'clef', 'd...","['surveillance', 'lieux', 'touristique', 'amen..."
1,Il faut donner des sacs poubelles et des cendr...,56,18,15,0.62,0.19,0.19,0,0,21,...,5,7,0,0,"['Il', 'faut', 'donner', 'des', 'sacs', 'poube...","['Min', 'Min', 'Min', 'Min', 'Min', 'Min', 'Mi...","['donner', 'des', 'sacs', 'poubelles', 'et', '...","['il', 'des', 'et', 'des', 'de', 'aux', 'des',...","['donner', 'sac', 'poubelle', 'cendrier', 'poc...","['donner', 'sac', 'poubelle', 'cendrier', 'poc..."
2,Il faut récompenser et mettre en avant les act...,35,7,14,0.62,0.13,0.25,0,5,12,...,1,5,0,0,"['Il', 'faut', 'récompenser', 'et', 'mettre', ...","['Min', 'Min', 'Min', 'Min', 'Min', 'Min', 'Mi...","['recompenser', 'et', 'mettre', 'en', 'avant',...","['il', 'et', 'en', 'les', 'des', 'du', 'et', '...","['recompenser', 'mettre', 'avant', 'acte', 'pr...","['recompenser', 'mettre', 'avant', 'acte', 'pr..."


## Première representation en nuage de mots
### Regroupement de tous les tokens

In [3]:
# 🧹 Étape 1 : Corriger les éléments mal typés dans la colonne 'tokens_final'
# Si une cellule contient une chaîne de caractères (str), on la convertit en vraie liste Python
df["tokens_final"] = df["tokens_final"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
# 👉 Exemple :
# Si x == "[ 'surveillance', 'lieux' ]" (une chaîne de texte)
# alors ast.literal_eval(x) => ['surveillance', 'lieux'] (vraie liste Python)
# Cela évite que les chaînes soient mal interprétées comme une suite de caractères.

In [4]:
# 📥 Étape 2 : Extraire la colonne entière sous forme de liste de listes
liste_de_listes = df["tokens_final"].tolist()
# 👉 Résultat :
# [['surveillance', 'lieux'], ['espaces', 'publics'], ['nature'], ...]

In [5]:
# 🧵 Étape 3 : Aplatir toutes les sous-listes en une seule liste
liste_plate = list(chain.from_iterable(liste_de_listes))
# 👉 Résultat :
# ['surveillance', 'lieux', 'espaces', 'publics', 'nature', ...]

# À ce stade, 'liste_plate' contient tous les mots (tokens) de tout ton corpus, prêts à être analysés.

In [6]:
liste_plate[0:10]

['surveillance',
 'lieux',
 'amende',
 'clef',
 'degradation',
 'alimentaire',
 'autre',
 'salissure',
 'gribouillage',
 'donner']

In [7]:
# Concaténer tous les tokens en une seule chaîne de caractères
texte = " ".join(liste_plate)

### Création d'un mask pour obtenir la forme de l'Ille-et-Vilaine

In [8]:
# Ouvrir l'image en noir et blanc
img = Image.open("../src/img/departement-ille-et-vilaine_nb.png")

# Agrandir l'image
# LANCZOS = interpolation haute qualité
img = img.resize((1600, 1600), Image.LANCZOS)

# Création du mak a partir de l'image agrandie
mask = np.array(img.convert("L"))  # Convertir en noir et blanc

### Création de l'objet nuage de mots

In [9]:
# Créer un objet WordCloud  background_color='white'
wc = WordCloud(
    font_path="../src/font/Light.ttf",
    mask=mask,
    colormap="Greens",
    margin=0,
)  # Thème de couleur en vert dégradé
wc.generate(texte)

In [10]:
# Convertion du nuage de mots (numpy array)
img_array = wc.to_array()

In [11]:
# Affichage de la figure avec Plotly
fig = px.imshow(img_array)
fig.update_layout(
    width=1000,
    height=1030,
    margin=dict(l=0, r=0, t=30, b=0),
    title=dict(
        text="Le tourisme vert en Ille-et-Vilaine",
        font=dict(size=25, color="black", family="Arial"),
        xanchor="center",
        yanchor="top",
        x=0.5,
        y=0.995,
    ),
)

fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)

fig.show()

## Sauvegarde de la figure

In [12]:
# Sauvegarder le nuage de mots dans un fichier
wc.to_file("../outputs/Le tourisme vert en Ille-et-Vilaine.png")

## Vectorisation avec Word2Vec

Nous allons tester quatre versions du modèle Word2Vec :

- **CBOW** :
  - avec les tokens juste après la lemmatisation (comprenant les stop words)
  - avec les tokens sans stop words

- **Skip-Gram** :
  - avec les tokens juste après la lemmatisation (comprenant les stop words)
  - avec les tokens sans stop words

### Comment paramétrer le modèle ?

- **`window`** : correspond à la taille de la fenêtre de contexte autour d’un mot. Je la paramètre en utilisant la **longueur moyenne des listes de tokens**.
- **`vector_size`** : taille des vecteurs de sortie. Elle doit être définie en testant plusieurs valeurs et en interprétant les mots les plus similaires. Cela dépend fortement de la taille du corpus.  
  → Dans notre cas, on dispose de **moins de 1500 propositions**, contenant environ **10 mots chacune**. Il est donc recommandé d’utiliser des vecteurs de **taille modeste** (par exemple entre 50 et 100).
- **`sg`** : définit l’architecture :
  - `0` → CBOW (Continuous Bag of Words)
  - `1` → Skip-Gram
- **`hs`** : méthode d’optimisation :
  - `1` → Hierarchical Softmax (souvent utilisé avec CBOW)
  - `0` → Négative Sampling (souvent utilisé avec Skip-Gram)

In [13]:
df["tokens_lemmatized"] = df["tokens_lemmatized"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# Calculer la longueur moyenne des listes dans la colonne 'tokens_lemmatized'
average_length = df["tokens_lemmatized"].apply(len).mean()
average_length

9.784326858673811

### Version C-BOW
<img src="../src/img/CBOW.png" alt="CBOW" style="width: 70%;">

#### Avec stop-words

In [14]:
cbow_w2v_with_stop_words = Word2Vec(
    sentences=df["tokens_lemmatized"], vector_size=20, sg=0, hs=1, window=5
)

#### Création d'une fonction pour tester quelques similarité

In [15]:
def get_similar_words_df(
    model: Word2Vec, word_list: list[str], topn: int = 10
) -> pd.DataFrame:
    """
    Returns a DataFrame containing the most similar words for each word in the input list.

    Args:
        model (Word2Vec): A trained Word2Vec model.
        word_list (list[str]): List of words to retrieve similar words for.
        topn (int): Number of similar words to return per input word (default is 10).

    Returns:
        pd.DataFrame: A DataFrame where each column corresponds to one word from word_list,
                      and contains its top-N most similar words.
    """
    results = {}

    for word in word_list:
        if word in model.wv:
            similar_words = model.wv.most_similar(word, topn=topn)
            results[word] = [w for w, _ in similar_words]
        else:
            results[word] = [None] * topn  # or use np.nan if preferred

    return pd.DataFrame(results)

#### Test de quelques similarités

In [16]:
words = [
    "bus",
    "train",
    "velo",
    "camping",
    "dechet",
    "nature",
    "ecologique",
    "responsable",
]
similar_df = get_similar_words_df(cbow_w2v_with_stop_words, words)
similar_df

,bus,train,velo,camping,dechet,nature,ecologique,responsable
0,taxer,plus,developper,bon,ligne,commun,velos,moyen
1,activite,long,touristique,afin,environnement,local,france,velo
2,voiture,itineraire,impact,prendre,eviter,tout,chaque,autre
3,mode,region,public,eco,exemple,arreter,voyager,developper
4,location,impact,permettre,centre,poubelle,favoriser,environnement,touristique
5,tourisme,local,commun,carbon,favoriser,inciter,prendre,public
6,creer,eau,autre,leur,touristique,bien,meme,impact
7,sans,inciter,moins,etre,nature,environnement,moyen,voyage
8,enfant,moins,tourisme,offrir,solution,permettre,permettre,vehicule
9,conscience,commun,responsable,tourisme,inciter,autre,developper,plus


#### Même chose sans stop-words

In [17]:
cbow_w2v_without_stop_words = Word2Vec(
    sentences=df["tokens_final"], vector_size=20, sg=0, hs=1, window=5
)

In [18]:
similar_df = get_similar_words_df(cbow_w2v_without_stop_words, words)
similar_df

,bus,train,velo,camping,dechet,nature,ecologique,responsable
0,partir,arreter,train,conscience,offrir,permettre,carbon,france
1,entreprise,transport,proposer,court,respecter,hebergement,bon,hebergement
2,polluant,velo,place,velos,polluant,sejour,rendre,site
3,national,bon,prix,petit,bon,creer,ville,territoire
4,acce,region,promouvoir,place,avion,tre,possible,plage
5,engager,ville,bon,ville,developper,entrer,etc,sejour
6,grand,territoire,conscience,avion,habitant,site,environnement,avion
7,interdir,proposer,camping,etre,limiter,responsable,toilette,activite
8,arreter,place,court,territoire,utilisation,action,site,long
9,interieur,plage,francai,vie,accueil,territoire,train,personne


#### Sauvegarde des versions CBOW

In [19]:
if not os.path.exists("../outputs/WE_models"):
    os.mkdir("../outputs/WE_models")

In [20]:
for i, cbow in enumerate([cbow_w2v_with_stop_words, cbow_w2v_without_stop_words]):
    # Save the model
    cbow.save(f"../outputs/WE_models/cbow_20D_{i}")

### Version Skip-Gram
<img src="../src/img/Skip-Gram.png" alt="Skip-Gram" style="width: 70%;">

#### Avec stop-words

In [21]:
skgram_w2v_with_stop_words = Word2Vec(
    sentences=df["tokens_lemmatized"], vector_size=20, sg=1, hs=1, window=5
)

In [22]:
similar_df = get_similar_words_df(skgram_w2v_with_stop_words, words)
similar_df

,bus,train,velo,camping,dechet,nature,ecologique,responsable
0,leger,trajet,commun,sonore,proche,eduquer,connaitre,fin
1,vouloir,long,utilisation,masse,respecter,bien,organisation,culturel
2,covoiturage,ferroviaire,covoiturage,etre,centre,aupre,meme,professionnel
3,deplacer,velo,energie,critere,eduquer,participer,tourist,comme
4,accueillir,moins,navette,contre,decouverte,monde,voyageur,tourisme
5,location,route,publicite,souvent,informer,environnement,engagement,transition
6,multiplier,vol,alternative,littoral,respect,age,culturel,voyage
7,navette,ligne,remettre,verbaliser,age,jeune,chasse,francai
8,seulement,energie,duree,reduit,proteger,flor,strategie,proposer
9,sans,billet,pied,passer,restaurateur,campagn,former,celui


#### Sans stop-words

In [23]:
skgram_w2v_without_stop_words = Word2Vec(
    sentences=df["tokens_final"], vector_size=20, sg=1, hs=1, window=5
)

In [24]:
similar_df = get_similar_words_df(skgram_w2v_without_stop_words, words)
similar_df

,bus,train,velo,camping,dechet,nature,ecologique,responsable
0,depart,boire,doux,modele,lieux,simple,voyageur,repartir
1,entreprise,moyen,boire,systeme,respecter,alimentaire,saison,geste
2,appliquer,typ,vert,cesser,structure,pedestre,animal,pro
3,restauration,depart,reduction,celer,climatique,point,informer,confort
4,bateau,velo,train,prevoir,travailler,confort,etc,education
5,etranger,mode,promouvoir,mise,proteger,responsable,masse,echange
6,tarif,billet,notamment,prise,entretien,guide,secteur,an
7,dont,faciliter,deplacement,enfant,preservation,age,logement,pari
8,prestataire,periode,francai,sonore,eviter,tre,campagn,equipement
9,periode,transport,alternative,commune,route,chez,usager,impliquer


#### Sauvegarde des versions Skip-Gram

In [ ]:
for i, skgram in enumerate([skgram_w2v_with_stop_words, skgram_w2v_without_stop_words]):
    # Save the model
    skgram.save(f"../outputs/WE_models/skgram_20D_{i}")